In [ ]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict,train_test_split
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer




In [ ]:
d=pd.read_csv('/content/accepted_2007_to_2018Q4.csv')

/tmp/ipython-input-352872338.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  d=pd.read_csv('/content/accepted_2007_to_2018Q4.csv')


In [ ]:
data=d[["loan_amnt","term","int_rate", "installment","grade","sub_grade","fico_range_low","annual_inc","dti","emp_length","purpose","open_acc","total_acc","revol_util","delinq_2yrs",
"inq_last_6mths","loan_status"]]
data.dropna(subset=['loan_status'], inplace=True)
print(data.head())

   loan_amnt        term  int_rate  installment grade sub_grade  \
0     3600.0   36 months     13.99       123.03     C        C4   
1    24700.0   36 months     11.99       820.28     C        C1   
2    20000.0   60 months     10.78       432.66     B        B4   
3    35000.0   60 months     14.85       829.90     C        C5   
4    10400.0   60 months     22.45       289.91     F        F1   

   fico_range_low  annual_inc    dti emp_length             purpose  open_acc  \
0           675.0     55000.0   5.91  10+ years  debt_consolidation       7.0   
1           715.0     65000.0  16.06  10+ years      small_business      22.0   
2           695.0     63000.0  10.78  10+ years    home_improvement       6.0   
3           785.0    110000.0  17.06  10+ years  debt_consolidation      13.0   
4           695.0    104433.0  25.37    3 years      major_purchase      12.0   

   total_acc  revol_util  delinq_2yrs  inq_last_6mths loan_status  
0       13.0        29.7          0.0     

/tmp/ipython-input-560159703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['loan_status'], inplace=True)


In [ ]:
X=data.drop('loan_status',axis=1)
y=data['loan_status']

In [ ]:
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(
        handle_unknown='ignore',
        sparse_output=True,
        dtype='int8'
    ))
])
ct = ColumnTransformer(
    transformers=[
        ('num', num_pipeline,
         make_column_selector(dtype_include=(int, float))),
        ('cat', cat_pipeline,
         make_column_selector(dtype_include=object))
    ]
)

In [ ]:
X=ct.fit_transform(X)


In [ ]:
y=((data['loan_status']=="Fully Paid").astype(int))
y.head()

,loan_status
0,1
1,1
2,1
3,0
4,1


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_test.shape

(3366, 78)

In [ ]:
rf = RandomForestClassifier(
    random_state=42,
    n_jobs=2
)
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [80, 120],
    'max_depth': [8, 10, 12],
    'min_samples_leaf': [5, 10],
    'max_features': ['sqrt', 0.5],
    'class_weight': [{0:1, 1:1.5}, {0:1, 1:2}]
}

rs = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=5,
    scoring='recall',
    cv=3,
    n_jobs=2,
    verbose=2,
    random_state=42
)

rs.fit(X_train, y_train)


Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_jobs=2, random_state=42),
                   n_iter=5, n_jobs=2,
                   param_distributions={'class_weight': [{0: 1, 1: 1.5},
                                                         {0: 1, 1: 2}],
                                        'max_depth': [8, 10, 12],
                                        'max_features': ['sqrt', 0.5],
                                        'min_samples_leaf': [5, 10],
                                        'n_estimators': [80, 120]},
                   random_state=42, scoring='recall', verbose=2)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
y_pred=rs.predict(X_test)
y_probas=rs.predict_proba(X_test)


for t in np.arange(0.5, 0.95, 0.05):

    y_pred_t = (y_probas[:, 1] >= t).astype(int)
    print(f"\nThreshold = {t:.2f}")
    print(classification_report(y_test, y_pred_t))



Threshold = 0.50
              precision    recall  f1-score   support

           0       0.55      0.01      0.02       970
           1       0.71      1.00      0.83      2396

    accuracy                           0.71      3366
   macro avg       0.63      0.50      0.43      3366
weighted avg       0.67      0.71      0.60      3366


Threshold = 0.55
              precision    recall  f1-score   support

           0       0.68      0.13      0.22       970
           1       0.74      0.97      0.84      2396

    accuracy                           0.73      3366
   macro avg       0.71      0.55      0.53      3366
weighted avg       0.72      0.73      0.66      3366


Threshold = 0.60
              precision    recall  f1-score   support

           0       0.66      0.35      0.46       970
           1       0.78      0.93      0.85      2396

    accuracy                           0.76      3366
   macro avg       0.72      0.64      0.65      3366
weighted avg       0

Since false positives are more costly in loan approval, I selected a threshold of 0.80 which provides high precision for fully paid loans while still maintaining strong recall and default detection.”